# AI-Generated Zoology

## Import TensorFlow and other libraries

In [1]:
import os
import time
import tensorflow as tf
import matplotlib.pyplot as plt
from IPython.display import clear_output
tf.__version__ 

'2.1.0'

## Load the dataset

## Input Pipeline

## Build the Generator

    

## Build the Discriminator


## Define the loss functions and the optimizer

* **Discriminator loss**

  
* **Generator loss**


## Checkpoints (Object-based saving)

## Training


## Generate Images


## Restore the latest checkpoint and test

## Testing on the entire test dataset